# D1 - Walvisualisatie
Noor Koevoet, Glenn de Graaff, Anne Beks, Job Kolhorn

D1

## Introductie
Persvrijheid is een principe dat stelt dat de pers vrij is om te schrijven, zeggen en publiceren wat zij willen. De overheid is in beginsel verboden om censuur toe te passen en mag niet vooraf controleren wat wel of niet gezegd mag worden. In Nederland is persvrijheid, samen met het recht op meningsuiting, vastgelegd in artikel 7 van de Grondwet. Ook in andere landen en binnen Europees verband zijn er wetten die persvrijheid en vrijheid van meningsuiting beschermen.

Om te verzekeren dat journalisten hun werk veilig kunnen doen, hebben zij een beschermde status van het Europese Hof voor de Rechten van de Mens (EHRM). Journalisten brengen informatie naar buiten over algemeen belangrijke zaken en zijn daarom van cruciaal belang voor de democratie. Hoewel er vrijheid is om te publiceren wat men wil, betekent dit niet dat dit onbestraft kan gebeuren. Wanneer publicaties discriminerend zijn of aanzetten tot opruiing, kan iemand aangeklaagd en veroordeeld worden. Dit gebeurt echter pas achteraf; vóór publicatie is censuur niet toegestaan.

In veel landen is persvrijheid echter niet gegarandeerd. In landen met een autocratisch regime is de vrijheid van meningsuiting vaak beperkt of volledig afwezig. Ook in landen waar conflicten gaande zijn, zijn er mogelijk meerdere machten aan de leiding, of lijkt er niemand aan de macht te zijn. In zulke situaties is het moeilijk om te waarborgen dat de wet wordt nageleefd.

Deze data-analyse zal onderzoeken of de mate van persvrijheid wordt beïnvloed door conflicten of de mate van democratie. Door gebruik te maken van data van Reporters Without Borders (RSF), The Armed Conflict Location & Event Data Project (ACLED) en Our World in Data, zullen we scores en ranglijsten analyseren rondom persvrijheid, democratie en conflicten wereldwijd. Het perspectief dat conflicten de persvrijheid verminderen en het perspectief dat autocratische bestuursvormen de persvrijheid verminderen, worden hier onderzocht.

[Mensenrechten.nl](https://www.mensenrechten.nl/themas/vrijheid-van-meningsuiting)


## Dataset and Preprocessing

Tekst

In [266]:
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact

import numpy as np

from IPython.display import display

init_notebook_mode(connected=True)

### Persvrijheid dataset

Voor de persvrijheid data maken we gebruik van de datasets van Reporters Without Borders (RSF). Deze datasets bevatten informatie over de persvrijheid in verschillende landen, waaronder de persvrijheidsscore en de ranglijst van elk land. De datasets die wij gebruiken zijn van de jaren 2019 tot en met 2024.

Bron: [RSF.org](https://rsf.org/en/index)

Deze datasets worden eerst ingeladen en daarna waar nodig een aantal kolommen hernoemd. Kolommen zoals de de namen van landen in verschillende landen worden verwijder, omdat wij in deze opdracht alleen de Engelse namen van landen gebruiken. Uiteindelijk worden alle datasets samengevoegd tot een dataframe, deze dataframe wordt gebruikt voor de rest van de analyses bij persvrijheid.

#### Datasets samenvoegen

De datasets van 2022 tot en met 2024 hebben meer data dan de datasets van 2019 tot en met 2021. Eerst worden de oude datasets gecombineerd en de nieuwe datasets gecombineerd.

##### Oude datasets

In [267]:
press_freedom_old = pd.concat([
    pd.read_csv('./data/persvrijheid/2019.csv', sep=';'),
    pd.read_csv('./data/persvrijheid/2020.csv', sep=';'),
    pd.read_csv('./data/persvrijheid/2021.csv', sep=';'),
])

# Rename columns so they're easier to use.
press_freedom_old = press_freedom_old.rename(
    columns={
        "Year (N)": "year",
        "ISO": "iso",
        "EN_country": "country",
        "Rank N": "rank",
        "Score N": "score",
    })

# Remove columns that are not needed.
press_freedom_old = press_freedom_old.drop(columns=[
    "Score N without the exactions",
    "Score N with the exactions",
    "Rank N-1",
    "Rank evolution",
    "Score exactions",
    "Score N-1",
    "Zone", # Need to drop this, since data in this column from 2022 is different than everywhere else.
])

# Remove all other country names except 'country'.
press_freedom_old = press_freedom_old.drop(columns=[
    col
    for col in press_freedom_old.columns
    if "country" in col.lower() and col != "country"
])

display(press_freedom_old.head())


,year,iso,rank,score,country
0,2019,NOR,1,"92,18",Norway
1,2019,FIN,2,"92,1",Finland
2,2019,SWE,3,"91,69",Sweden
3,2019,NLD,4,"91,37",Netherlands
4,2019,DNK,5,"90,13",Denmark


##### Nieuwe datasets

In [268]:
press_freedom_new = pd.concat([
    pd.read_csv('./data/persvrijheid/2022.csv', sep=';'),
    pd.read_csv('./data/persvrijheid/2023.csv', sep=';'),
    pd.read_csv('./data/persvrijheid/2024.csv', sep=';'),
])

# Rename columns so they're easier to use.
press_freedom_new = press_freedom_new.rename(
    columns={
        "Year (N)": "year",
        "ISO": "iso",
        "Country_EN": "country",
        "Rank": "rank",
        "Score": "score",
    })

# Remove columns that are not needed.
press_freedom_new = press_freedom_new.drop(columns=[
    "Political Context", "Rank_Pol",
    "Economic Context", "Rank_Eco",
    "Legal Context", "Rank_Leg",
    "Social Context", "Rank_Soc",
    "Safety", "Rank_Saf",
    "Rank N-1",
    "Rank evolution",
    "Score N-1",
    "Score evolution",
    "Situation",
    "Zone", # Need to drop this, since data in this column from 2022 is different than everywhere else.
])

# Remove all other country names except 'country'.
press_freedom_new = press_freedom_new.drop(columns=[
    col
    for col in press_freedom_new.columns
    if "country" in col.lower() and col != "country"
])

display(press_freedom_new.head())

,iso,score,rank,country,year
0,NOR,"92,65",1,Norway,2022
1,DNK,"90,27",2,Denmark,2022
2,SWE,"88,84",3,Sweden,2022
3,EST,"88,83",4,Estonia,2022
4,FIN,"88,42",5,Finland,2022


##### Datasets combineren

In [269]:
press_freedom_df = pd.concat([press_freedom_old, press_freedom_new])

display(press_freedom_df)

,year,iso,rank,score,country
0,2019,NOR,1,"92,18",Norway
1,2019,FIN,2,"92,1",Finland
2,2019,SWE,3,"91,69",Sweden
3,2019,NLD,4,"91,37",Netherlands
4,2019,DNK,5,"90,13",Denmark
...,...,...,...,...,...
175,2024,IRN,176,"21,3",Iran
176,2024,PRK,177,"20,66",North Korea
177,2024,AFG,178,"19,09",Afghanistan
178,2024,SYR,179,"17,41",Syria


### Democratie dataset

Voor de democratie data maken we gebruik van de datasets van Our World in Data. Deze datasets bevatten informatie over de democratie in verschillende landen, waaronder de democratie score en de ranglijst van elk land. De datasets die wij gebruiken heeft data vanaf het jaar 2019 tot en met 2023.

Bron: [OurWorldInData.org](https://ourworldindata.org/grapher/democracy-index-eiu?tab=table&time=earliest..2023)

Deze datasets worden eerst ingeladen en daarna waar nodig een aantal kolommen hernoemd. Uiteindelijk worden alle datasets samengevoegd tot een dataframe, deze dataframe wordt gebruikt voor de rest van de analyses bij democratie.

In [271]:
democracy_df = pd.read_csv('./data/democracy.csv')

# Rename columns so they're easier to use.
democracy_df = democracy_df.rename(
    columns={
        "Entity": "country",
        "Code": "iso",
        "Year": "year",
        "Democracy score": "score",
    })

# Remove data from years previous to 2019.
democracy_df = democracy_df[democracy_df["year"] >= 2019]

#### Democratie ranken

Ieder land krijgt een ranking gebaseerd op de democratie score. Landen met een hogere score krijgen een lagere ranking. Landen met dezelfde score krijgen dezelfde ranking. Per jaar wordt de ranking van de landen opnieuw berekend.

In [272]:
def rank_countries_per_year(data: pd.DataFrame) -> pd.DataFrame:
    new_data = data.copy()
    
    # Get a sorted list of unique scores for each year.
    unique_scores = {
        year: sorted(
            list(new_data[new_data["year"] == year]["score"].unique()),
            reverse=True,
        )
        for year in new_data["year"].unique()
    }

    # Calculate the ranking for each country based on the unique scores from that year.
    new_data["ranking"] = new_data.apply(
        lambda x: unique_scores[x["year"]].index(x["score"]) + 1,
        axis=1,
    )
    
    return new_data

democracy_df = rank_countries_per_year(democracy_df)
democracy_df = democracy_df.sort_values(
    by=["year", "score"],
    ascending=False
).reset_index().drop(columns=["index"])

#### Persvrijheid en Democratie data combineren

De data van de democratie en persvrijheid worden samengevoegd tot één dataframe. Deze dataframe wordt gebruikt voor de rest van de analyses.

In [273]:
dem_press_df = pd.merge(
    democracy_df,
    press_freedom_df,
    on=['iso', "year"],
    how='inner',
    suffixes=("_dem", "_press")
)

dem_press_df = dem_press_df.drop(columns=[
    "country_press",
])

dem_press_df = dem_press_df.rename(columns={
    "country_dem": "country",
    "score_dem": "democracy_score",
    "score_press": "press_freedom_score",
    "ranking": "democracy_ranking",
    "rank": "press_freedom_ranking",
})

display(dem_press_df)

,country,iso,year,democracy_score,democracy_ranking,press_freedom_ranking,press_freedom_score
0,Norway,NOR,2023,9.81,1,1,"95,18"
1,New Zealand,NZL,2023,9.61,2,13,"84,23"
2,Iceland,ISL,2023,9.45,3,18,"83,19"
3,Sweden,SWE,2023,9.39,4,4,"88,15"
4,Finland,FIN,2023,9.30,5,5,"87,94"
...,...,...,...,...,...,...,...
811,Chad,TCD,2019,1.61,151,122,"63,29"
812,Syria,SYR,2019,1.43,152,174,"28,22"
813,Central African Republic,CAF,2019,1.32,153,145,"52,73"
814,Democratic Republic of Congo,COD,2019,1.13,154,154,"48,29"


### Conflict dataset

Voor de conflict data maken we gebruik van de datasets van The Armed Conflict Location & Event Data Project (ACLED). Deze datasets bevatten informatie over conflicten in verschillende landen, waaronder de conflict index ranking van elk land. De dataset die wij gebruiken heeft data vanaf het jaar 2023, en vergelijkt de data van een land met de data van het land uit 2019.

Bron: [ACLEDdata.com](https://acleddata.com/conflict-index/#download)

Deze dataset wordt eerst ingeladen en daarna waar nodig een aantal kolommen hernoemd. Uiteindelijk wordt de dataset gebruikt voor de rest van de analyses bij conflicten.

In [324]:
conflict_df = pd.read_csv('./data/conflict.csv')

# Rename columns so they're easier to use.
conflict_df = conflict_df.rename(columns={
    "Index Ranking 2023": "conflict_ranking",
    "Index Category 2023": "conflict_category",
    "Change in ranking since 2019": "conflict_ranking_change",
})

conflict_df = conflict_df[[
    'country',
    'conflict_ranking',
    "conflict_category",
    "conflict_ranking_change",
]]

conflict_df["conflict_ranking_2019"] = conflict_df["conflict_ranking"] + conflict_df["conflict_ranking_change"]

display(conflict_df.columns)
display(conflict_df.head())

Index(['country', 'conflict_ranking', 'conflict_category',
       'conflict_ranking_change', 'conflict_ranking_2019'],
      dtype='object')

,country,conflict_ranking,conflict_category,conflict_ranking_change,conflict_ranking_2019
0,Myanmar,1,Extreme,17,18
1,Syria,2,Extreme,-1,1
2,Palestine,3,Extreme,20,23
3,Mexico,4,Extreme,-1,3
4,Nigeria,5,Extreme,6,11


In [335]:
# Give the countries a color based on their conflict ranking
def determine_color(conflict_rank: int):
    if conflict_rank <= 10:
        return 'red'
    elif conflict_rank <= 30:
        return 'orange'
    elif conflict_rank <= 50:
        return 'yellow'
    else:
        return 'blue'

conflict_df['color'] = conflict_df['conflict_ranking'].apply(determine_color)

display(conflict_df.head())

,country,conflict_ranking,conflict_category,conflict_ranking_change,conflict_ranking_2019,color
0,Myanmar,1,Extreme,17,18,red
1,Syria,2,Extreme,-1,1,red
2,Palestine,3,Extreme,20,23,red
3,Mexico,4,Extreme,-1,3,red
4,Nigeria,5,Extreme,6,11,red


#### Persvrijheid en Conflict data combineren

In [336]:
press_conflict_df = pd.merge(
    press_freedom_df[press_freedom_df["year"] == 2023],
    conflict_df,
    on=['country'],
    how='inner',
    suffixes=("_press", "_conflict")
)

press_conflict_df = press_conflict_df.rename(columns={
    "score": "press_freedom_score",
    "rank": "press_freedom_ranking",
    "color": "conflict_color",
})

press_conflict_df.drop(columns=[
    "year",
])

display(press_conflict_df)

,year,iso,press_freedom_ranking,press_freedom_score,country,conflict_ranking,conflict_category,conflict_ranking_change,conflict_ranking_2019,conflict_color
0,2023,NOR,1,"95,18",Norway,146,Low/Inactive,-13,133,blue
1,2023,IRL,2,"89,91",Ireland,140,Low/Inactive,-7,133,blue
2,2023,DNK,3,"89,48",Denmark,163,Low/Inactive,-30,133,blue
3,2023,SWE,4,"88,15",Sweden,140,Low/Inactive,-7,133,blue
4,2023,FIN,5,"87,94",Finland,146,Low/Inactive,-13,133,blue
...,...,...,...,...,...,...,...,...,...,...
163,2023,TKM,176,"25,82",Turkmenistan,163,Low/Inactive,-69,94,blue
164,2023,IRN,177,"24,81",Iran,46,Turbulent,9,55,yellow
165,2023,VNM,178,"24,58",Vietnam,111,Low/Inactive,-4,107,blue
166,2023,CHN,179,"22,97",China,66,Low/Inactive,-20,46,blue


#### Democratie en Conflict data combineren

In [337]:
dem_conflict_df = pd.merge(
    democracy_df[democracy_df["year"] == 2023],
    conflict_df,
    on=['country'],
    how='inner',
    suffixes=("_dem", "_conflict")
)

dem_conflict_df = dem_conflict_df.rename(columns={
    "score": "democracy_score",
    "ranking": "democracy_ranking",
    "color": "conflict_color",
})

display(dem_conflict_df)

,country,iso,year,democracy_score,democracy_ranking,conflict_ranking,conflict_category,conflict_ranking_change,conflict_ranking_2019,conflict_color
0,Norway,NOR,2023,9.81,1,146,Low/Inactive,-13,133,blue
1,New Zealand,NZL,2023,9.61,2,163,Low/Inactive,-30,133,blue
2,Iceland,ISL,2023,9.45,3,146,Low/Inactive,-13,133,blue
3,Sweden,SWE,2023,9.39,4,140,Low/Inactive,-7,133,blue
4,Finland,FIN,2023,9.30,5,146,Low/Inactive,-13,133,blue
...,...,...,...,...,...,...,...,...,...,...
138,Russia,RUS,2023,2.22,139,36,Turbulent,19,55,yellow
139,Guinea,GIN,2023,2.21,140,65,Low/Inactive,-1,64,blue
140,Gabon,GAB,2023,2.18,141,105,Low/Inactive,12,117,blue
141,Burundi,BDI,2023,2.13,142,32,Turbulent,-12,20,yellow


## Conflicten Verminderen de Persvrijheid

### Argument 1

**Landen die in conflict zijn hebben op dat moment een lage persvrijheid.**

Het is algemeen bekend dat conflicten een negatieve impact hebben op verschillende aspecten van de samenleving, hier is persvrijheid ook een deel van. Tijdens conflicten wordt de veiligheid van journalisten vaak in gevaar gebracht, waardoor ze hun werk niet vrij kunnen uitvoeren. De dreiging van geweld en intimidatie zorgt ervoor dat journalisten zichzelf censureren of stoppen met rapporteren over gevoelige onderwerpen. De pie chart hieronder biedt een helder inzicht in de verdeling van conflictniveaus wereldwijd. Uit het diagram blijkt dat 71,4% van de landen momenteel een lage of inactieve conflictstatus heeft. Deze landen ervaren doorgaans minder ernstige beperkingen op persvrijheid. In tegenstelling hiermee bevindt de overige 28,6% van de landen zich in een hoge conflictstatus, waar persvrijheid mogelijk wordt ingeperkt. Dit betekent dat bijna een derde van de journalisten in deze landen te maken heeft met aanzienlijke uitdagingen en belemmeringen bij hun werk.

In [281]:
color_counts = press_conflict_df['conflict_color'].value_counts()

color_labels = {
    'red': 'Extreem',
    'orange': 'Hoog',
    'yellow': 'Turbulent',
    'blue': 'Laag/Inactief'
}


labels = [color_labels[color] for color in color_counts.index]


fig = go.Figure(data=[go.Pie(
    labels = labels,
    values = color_counts.values,
    marker = dict(colors = color_counts.index)
)])

fig.update_layout(
    title = 'Aantal landen met bepaalde conflict status (index level) ',
    legend_title = 'Index level'
)

fig.show()

In [282]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=press_conflict_df['conflict_ranking'],
    y=press_conflict_df['press_freedom_ranking'],
    mode='markers',
    text=press_conflict_df['country'],
    marker=dict(
        color=press_conflict_df['conflict_color'],
        size=10
    ),
    name='Land',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='red', size=10),
    legendgroup='group1',
    showlegend=True,
    name='Extreem (Top 10)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='orange', size=10),
    legendgroup='group2',
    showlegend=True,
    name='Hoog (11-30)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='yellow', size=10),
    legendgroup='group3',
    showlegend=True,
    name='Turbulent (31-50)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='blue', size=10),
    legendgroup='group4',
    showlegend=True,
    name='Laag/inactief (>50)'
))

fig.add_trace(go.Scatter(
    x=[min(press_conflict_df['conflict_ranking']), max(press_conflict_df['conflict_ranking'])],
    y=[max(press_conflict_df['press_freedom_ranking']), min(press_conflict_df['press_freedom_ranking'])],
    mode='lines',
    line=dict(color='black'),
    name='Trend lijn'
))

fig.update_layout(
    title='Press Freedom Index Ranking vs Conflict Index Ranking (2023)',
    xaxis_title='Conflict Index Ranking (Index Ranking 2023)',
    yaxis_title='Press Freedom Index Ranking (Rank)',
    showlegend=True,
    height=700
)

fig.show()

correlation = press_conflict_df["conflict_ranking"].corr(press_conflict_df["press_freedom_ranking"])

print(f"Correlatie tussen de conflict index ranking en de persvrijheid index ranking: {correlation}")

Correlatie tussen de conflict index ranking en de persvrijheid index ranking: -0.42573893484944786


De analyse van de dataset suggereert dat conflicten een negatieve impact hebben op de persvrijheid. We hebben een significante negatieve correlatie gevonden tussen de Conflict Index Ranking en de Press Freedom Index Ranking voor het jaar 2023. Specifiek toont de visualisatie een correlatie van `-0.43` tussen deze twee indexen. Dit betekent dat naarmate een land hoger scoort op de Conflict Index Ranking (wat wijst op een lagere mate van conflict), het doorgaans lager scoort op de Press Freedom Index Ranking (wat wijst op meer persvrijheid).


### Argument 2

**Naarmate een conflict escaleert, verslechtert de persvrijheid nog meer.**

De relatie tussen conflicten en persvrijheid is complex en vertoont vaak zorgwekkende trends. Wanneer conflicten escaleren, lijkt de persvrijheid in samenlevingen significant te verslechteren. Dit fenomeen wordt goed geïllustreerd in gebieden zoals Palestina en Syrië. Gedurende de periode van 2019 tot 2023 is de Conflict Index Ranking voor Syrië relatief stabiel gebleven, in contrast hiermee heeft Palestina een opmerkelijke stijging in de Conflict Index Ranking doorgemaakt, van de 23e plaats in 2019 naar de 3e plaats in 2023, wat duidt op een aanzienlijke escalatie van het conflict in dat gebied.


In [334]:
data_conflict = pd.read_excel('./data/01-2024-conflic-index.xlsx')

data_conflict.rename(columns={'Country': 'country', 'Index Ranking 2023': 'Conflict_Rank_2023', 'Change in ranking since 2019': 'Change_Conflict'}, inplace=True)

# Merge the datasets for the press freedom index for 2019 and 2023
press_19_23_df = pd.merge(
    press_freedom_df[press_freedom_df['year'] == 2019][["country", "rank"]],
    press_freedom_df[press_freedom_df['year'] == 2023][["country", "rank"]],
    on='country',
    how='inner',
    suffixes=("_2019", "_2023")
)

# Calculate the change in ranking for the press freedom index between 2019 and 2023
press_19_23_df["change"] = press_19_23_df["rank_2023"] - press_19_23_df["rank_2019"]

data_conflict['Conflict_Rank_2019'] = data_conflict['Conflict_Rank_2023'] + data_conflict['Change_Conflict']

def filter_countries(text, dataset):
    if dataset == 'Conflict':
        return [
            country
            for country in conflict_df["country"]
            if text.lower() in country.lower()
        ]
    else:
        return [
            country
            for country in press_19_23_df["country"]
            if text.lower() in country.lower()
        ]


def plot_countries_ranking(countries):
    if not countries:
        print("Selecteer minstens een land")
        return
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('Conflict Index', 'Press Freedom Index'))
    
    for country in countries:

        selected_data_conflict = data_conflict[data_conflict['country'] == country]
        
        if len(selected_data_conflict) > 0:
            ranking_2019 = selected_data_conflict['Conflict_Rank_2019'].values[0]
            ranking_2023 = selected_data_conflict['Conflict_Rank_2023'].values[0]
            change = ranking_2023 - ranking_2019
            
            color = "Red" # Default color
            if change == 0:
                color = "blue"
            elif change > 0:
                color = "green"
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Conflict ({ranking_2019} -> {ranking_2023})'
            ), row=1, col=1)
        
        selected_press_df = press_19_23_df[press_19_23_df["country"] == country]
        
        if len(selected_press_df) > 0:
            ranking_2019 = selected_press_df["rank_2019"].values[0]
            ranking_2023 = selected_press_df["rank_2023"].values[0]
            change = selected_press_df["change"].values[0]
            
            color = "green" # Default color
            if change == 0:
                color = "blue"
            elif change > 0:
                color = "red"
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Press Freedom ({ranking_2019} -> {ranking_2023})'
            ), row=2, col=1)
        
    if len(countries) == 1:
        title = f'Verandering in Ranking voor {countries[0]}'
    elif len(countries) == 2:
        title = f'Verandering in Ranking voor {" en ".join(countries)}'
    elif len(countries) > 5:
        title = f'Verandering in Ranking voor {len(countries)} landen'
    else:
        title = f'Verandering in Ranking voor {", ".join(countries)}'
        
    fig.update_layout(
        height=800,
        title_text=title,
        showlegend=True,
    )
    
    fig.update_xaxes(title_text='Jaar', row=1, col=1)
    fig.update_xaxes(title_text='Jaar', row=2, col=1)
    fig.update_yaxes(title_text='Conflict Index Ranking', row=1, col=1)
    fig.update_yaxes(title_text='Press Freedom Index Ranking', row=2, col=1)
    
    fig.show()

dataset_dropdown = widgets.Dropdown(
    options=['Conflict', 'Press Freedom'],
    value='Conflict',
    description='Dataset:'
)

country_dropdown = widgets.SelectMultiple(
    options = data_conflict['country'].unique().tolist() if dataset_dropdown.value == 'Conflict' else press_19_23_df["country"].unique().tolist(),
    description = 'Landen:',
    disabled = False,
    layout = widgets.Layout(height='200px', width='400px'),
)

search_text = widgets.Text(
    value = '',
    placeholder = 'Zoek landen',
    description = 'Zoeken:',
    disabled = False
)

def handle_search(change):
    search_results = filter_countries(change.new, dataset_dropdown.value)
    
    selected_countries = list(country_dropdown.value)
    country_dropdown.options = search_results
    
    country_dropdown.value = [
        country
        for country in selected_countries
        if country in search_results
    ]

def update_country_dropdown(change):
    search_results = filter_countries(search_text.value, change.new)
    country_dropdown.options = search_results
    country_dropdown.value = []

search_text.observe(handle_search, names='value')
dataset_dropdown.observe(update_country_dropdown, names='value')

display(dataset_dropdown)
display(search_text)

interact(plot_countries_ranking, countries=country_dropdown)

Dropdown(description='Dataset:', options=('Conflict', 'Press Freedom'), value='Conflict')

Text(value='', description='Zoeken:', placeholder='Zoek landen')

interactive(children=(SelectMultiple(description='Landen:', layout=Layout(height='200px', width='400px'), opti…

<function __main__.plot_countries_ranking(countries)>

**DIT WERKT NOG NIET IN JUPYTER BOOK**

Hier is een foto van hoe het eruit moet zien:

![Visualisatie 1](./images/viz.png)

De visualisatie toont deze trends duidelijk. Het beeld schetst de mogelijke negatieve correlatie tussen de Conflict Index en de Press Freedom Index, waarbij landen met hogere conflictniveaus vaak lagere persvrijheidsscores vertonen. Dit suggereert dat journalisten in deze regio's te maken hebben met verhoogde risico's zoals geweld, intimidatie en censuur, wat hun mogelijkheid om vrij te rapporteren beperkt.

De data wijst erop dat tijdens conflicten vaak noodmaatregelen en censuur worden ingevoerd om de berichtgeving te controleren en de publieke opinie te beïnvloeden, wat verdere inperkingen van de persvrijheid met zich meebrengt. Hoewel deze correlatie aangeeft hoe conflicten de persvrijheid kunnen beïnvloeden, moeten we erkennen dat andere factoren zoals autoritaire regimes en economische instabiliteit ook een rol kunnen spelen. Desalniettemin benadrukt deze analyse het cruciale belang van het beschermen van journalisten en het handhaven van vrije media, vooral in tijden van crisis.


## Autocratische Bestuursvormen Verminderen de Persvrijheid

### Argument 1

**Landen met een meer autocratische bestuursvorm hebben een lagere persvrijheid.**

Het verband tussen bestuursvormen en persvrijheid is cruciaal binnen de context van democratie en mensenrechten. Autocratische regimes, die worden gekenmerkt door een sterke concentratie van macht en beperkte politieke vrijheden, vertonen vaak tendensen van onderdrukking van vrije media. Deze onderdrukking dient niet alleen om de macht te versterken, maar ook om elke vorm van oppositie streng te controleren en te beperken. Het resultaat hiervan is vaak censuur, arrestaties van journalisten en een strikte controle over mediakanalen, wat kan leiden tot lagere persvrijheidsscores in deze landen.

In [284]:
# creeer een scatterplot om de relatie tussen de democratie en persvrijheid aan te geven per land

fig = px.scatter(
    dem_press_df[dem_press_df["year"] == 2023],
    x='democracy_ranking',
    y='press_freedom_ranking',
    title = 'Democracy index ranking vs Press freedom index ranking 2023 ',
    hover_data=['country'],
    trendline="ols")

fig.update_layout(
    xaxis_title="Democracy index ranking 2023",
    yaxis_title="Press freedom index ranking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"

fig.show()

De visualisatie bestaat uit scatterplots die de relatie tussen de Democracy Index en de Press Freedom Index per land weergeven. Deze grafiek biedt een visuele representatie van de dynamiek tussen bestuursvorm en persvrijheid. Over het algemeen tonen landen met hogere scores op de Democracy Index, wat duidt op meer autocratische bestuursvormen, ook hogere scores op de Press Freedom Index, wat duidt op minder persvrijheid. Deze scatter plots demonstreren hoe de mate van democratie in een land samenhangt met de mate van persvrijheid die journalisten en mediaorganisaties daar ervaren.

### Argument 2

**Landen met een meer autocratische bestuursvorm hebben vaker conflicten.**

De twee pie charts hieronder bieden inzicht in de veranderingen van de Conflict Index Rankings van 2019 tot 2023. Uit de eerste pie chart blijkt dat `28%` van de landen hun Conflict Index Ranking heeft zien verslechteren gedurende deze periode. Tegelijkertijd toont de tweede pie chart dat in dezelfde periode `46.7%` van de landen een verslechtering heeft ervaren. Deze overlappende groep van landen, waarin zowel de Conflict Index Rankings als de persvrijheid mogelijk zijn verslechterd, wijst op een zorgwekkende trend van toenemende conflicten en mogelijke beperkingen van persvrijheid.

In [297]:
conditions = [
    (conflict_df['conflict_ranking_change'] > 0),
    (conflict_df['conflict_ranking_change'] < 0),
    (conflict_df['conflict_ranking_change'] == 0)
]
choices = ['Verslechterd', 'Verbeterd', 'Gelijk gebleven']

conflict_df['conflict_change_status'] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = conflict_df["conflict_change_status"].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Conflict Index Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

In [317]:
conditions = [
    (press_19_23_df["change"] > 0),
    (press_19_23_df["change"] < 0),
    (press_19_23_df["change"] == 0)
]
choices = ['Verbeterd', 'Verslechterd', 'Gelijk gebleven']
press_19_23_df["status"] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = press_19_23_df["status"].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Persvrijheidsindex Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

De scatterplot illustreert verder dat landen met meer autocratische bestuursvormen, gekenmerkt door hogere scores op de Democracy Index Ranking, doorgaans lagere scores hebben op de Conflict Index Ranking. Dit betekent dat autocratische regimes vaak een grotere neiging hebben tot conflicten, mogelijk als gevolg van beperkte politieke participatie, onderdrukking van oppositie en machtsmisbruik. Deze correlatie benadrukt de uitdagingen waarmee autocratische samenlevingen worden geconfronteerd in termen van zowel interne stabiliteit als externe betrekkingen.


## Andere Visualisaties

### Visualisatie 2

Creeer een line chart van de persvrijheid rankings per jaar. Gebruik ctrl+klik om meerdere landen te selecteren

You can also plot the data using static visualizations, such as the [seaborn](https://seaborn.pydata.org/#) library.

In [301]:
# Persvrijheid scores -  ctrl + klik om meerdere landen te selecteren

# Create a dropdown for country selection
country_dropdown = widgets.SelectMultiple(
    options= press_freedom_df["country"].unique(),
    value=[press_freedom_df["country"].unique()[0]],
    description='Country',
    disabled=False
)

# Function to update the line chart based on selected countries
def update_chart(selected_countries):
    filtered_df = press_freedom_df[press_freedom_df["country"].isin(selected_countries)]
    fig = px.line(
        filtered_df,
        x="year",
        y="rank",
        color="country", 
        labels={"rank": "Press Freedom Rank"},
        title='Persvrijheid ranking per land',
    )
    
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Persvrijheid rank"
    )
    fig.show()

# Create an interactive output
output = widgets.interactive_output(update_chart, {'selected_countries': country_dropdown})

# Display the dropdown and the output
display(country_dropdown, output)


SelectMultiple(description='Country', index=(0,), options=('Norway', 'Finland', 'Sweden', 'Netherlands', 'Denm…

Output()

### Visualisatie 3

Creeer een line chart van de democratie rankings per jaar. Gebruik ctrl+klik om meerdere landen te selecteren.

In [308]:
# democratie scores - ctrl + klik om meerdere landen te selecteren

# Create a dropdown for country selection
country_dropdown = widgets.SelectMultiple(
    options= democracy_df["country"].unique(),
    value=[democracy_df["country"].unique()[0]],
    description='Country',
    disabled=False
)

# Function to update the line chart based on selected countries
def update_chart(selected_countries):
    filtered_df = democracy_df[democracy_df["country"].isin(selected_countries)]
    
    fig = px.line(
        filtered_df,
        x='year',
        y='ranking',
        color='country', 
        labels={'score': 'Democracy Score'},
        title='Democratie ranking per land',
    )

    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Democratie rank"
    )

    fig.show()

# Create an interactive output
output = widgets.interactive_output(update_chart, {'selected_countries': country_dropdown})

# Display the dropdown and the output
display(country_dropdown, output)

SelectMultiple(description='Country', index=(0,), options=('Norway', 'New Zealand', 'Iceland', 'Sweden', 'Finl…

Output()

### Visualisatie 4

In [289]:
# creeer een scatterplot om de relatie tussen de democratie en conflict aan te geven per land

fig = px.scatter(
    dem_conflict_df,
    x='democracy_ranking',
    y='conflict_ranking',
    title = 'Democracy index ranking vs Conflict index ranking 2023',
    hover_data=['country'],
    trendline="ols",
)

fig.update_layout(
    xaxis_title="Democracy index ranking 2023",
    yaxis_title="Conflict index ranking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"


fig.show()

## Ter verduidelijking:


-----
- 'Conflict index':

Een hogere ranking (bijv. 1) geeft aan dat er in een land veel/grote conflicten zijn
Een lagere ranking (bijv. 150) geeft aan dat er in een land weinig/kleine conflicten zijn

--------
- 'Democratie index':

Een hogere ranking (bijv. 1) geeft aan dat er in een land een heersende democratie is
Een lagere ranking (bijv. 150) geeft aan dat er in een land geen democratie is

---------
- 'Persvrijheid  index':

Een hogere ranking (bijv. 1) geeft aan dat de pers in een land veel vrijheid heeft
Een lagere ranking (bijv. 150) geeft aan dat de pers in een land weinig vrijheid heeft.

## Reflectie

Tekst

## Work distribution

Tekst

## Referenties

Tekst

## Wereldkaart. Glenn kijk hier naar :)

In [ ]:
# laad de dataset
eind_df = pd.read_pickle('./data/dataset.pkl')

In [ ]:
# Creëer een wereldkaart om de verandering van rang wat betreft persvrijheid te laten zien per land (2019-2023)

# Maak een kleurenschaal
colorscale = [
    [0, '#9b0000'],          # Donker rood
    [54/(74+55), '#FF7377'], # Licht rood
    [55/(74+55), '#0000ff'], # Blauw
    [56/(74+55), '#90ee90'], # Licht groen
    [1, '#006400']           # Donker groen
]
 
# Maak de choropletenkaart 
trace = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['persvrijheid_verandering'],
    colorscale=colorscale,
    hovertemplate='<b>%{location}</b><br>Verandering in persvrijheid rang: %{z}<extra></extra>',
    colorbar=go.choropleth.ColorBar(
        x=0.5,
        y=0.92,
        xanchor='center',
        yanchor='bottom',
        orientation='h',
        len=0.5,
        thickness=10,
        tickmode='array',
        title='',
        tickvals=[eind_df['persvrijheid_verandering'].min(), 0, eind_df['persvrijheid_verandering'].max()],
        ticktext=['Verslechterd', 'Geen verandering', 'Verbeterd']
    ),
    zmin=eind_df['persvrijheid_verandering'].min(),
    zmax=eind_df['persvrijheid_verandering'].max(),
)
 
# Maak het figuur
fig = go.Figure(
    data=[trace],
    layout=go.Layout(
        width=790,
        height=640,
        title='Verandering in Persvrijheid Rang (2019-2023)',
        geo={'showocean': True, 'oceancolor': '#a8d5f2', 'landcolor': '#ffffff'},
        margin={'t': 80, 'r': 20, 'b': 80, 'l': 20},
    )
)
 

fig.update_geos(showcountries=True, showcoastlines=False)
 
fig.show()

In [ ]:
# Creëer een wereldkaart om de verandering van rang wat betreft conflict te laten zien per land (2019-2023)


# Maak een kleurenschaal
colorscale = [
    [0, '#9b0000'],            # Donker rood
    [112/(48+113), '#FF7377'], # Licht rood
    [113/(48+113), '#0000ff'], # Blauw
    [114/(48+113), '#90ee90'], # Licht groen
    [1, '#006400']             # Donker groen
]
 
# Maak de choropletenkaart
trace = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['conflict_verandering'],
    colorscale=colorscale,
    hovertemplate='<b>%{location}</b><br>Verandering in conflict rang: %{z}<extra></extra>',
    colorbar=go.choropleth.ColorBar(
        x=0.5,
        y=0.92,
        xanchor='center',
        yanchor='bottom',
        orientation='h',
        len=0.5,
        thickness=10,
        tickmode='array',
        title='',
        tickvals=[eind_df['conflict_verandering'].min(), 0, eind_df['conflict_verandering'].max()],
        ticktext=['Verslechterd', 'Geen verandering', 'Verbeterd']
    ),
    zmin=eind_df['conflict_verandering'].min(),
    zmax=eind_df['conflict_verandering'].max(),
)
 
# Maak het figuur
fig = go.Figure(
    data=[trace],
    layout=go.Layout(
        width=790,
        height=640,
        title='Verandering in Conflict Rang (2019-2023)',
        geo={'showocean': True, 'oceancolor': '#a8d5f2', 'landcolor': '#ffffff'},
        margin={'t': 80, 'r': 20, 'b': 80, 'l': 20},
    )
)
 
fig.update_geos(showcountries=True, showcoastlines=False)
 
fig.show()

In [ ]:
# Creëer een wereldkaart om de verandering van rang wat betreft democratie te laten zien per land (2019-2023)

# Maak een kleurenschaal
colorscale = [
    [0, '#9b0000'],          # Donker rood
    [34/(32+35), '#FF7377'], # Licht rood
    [35/(32+35), '#0000ff'], # Blauw voor 0
    [36/(32+35), '#90ee90'], # Licht groen
    [1, '#006400']           # Donker groen
]
 
# Maak de choropletenkaart
trace = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['democratie_verandering'],
    colorscale=colorscale,
    hovertemplate='<b>%{location}</b><br>Verandering in democratie rang: %{z}<extra></extra>',
    colorbar=go.choropleth.ColorBar(
        x=0.5,
        y=0.92,
        xanchor='center',
        yanchor='bottom',
        orientation='h',
        len=0.5,
        thickness=10,
        tickmode='array',
        title='',
        tickvals=[eind_df['democratie_verandering'].min(), 0, eind_df['democratie_verandering'].max()],
        ticktext=['Verslechterd', 'Geen verandering', 'Verbeterd']
    ),
    zmin=eind_df['democratie_verandering'].min(),
    zmax=eind_df['democratie_verandering'].max(),
)
 
# Maak het figuur
fig = go.Figure(
    data=[trace],
    layout=go.Layout(
        width=790,
        height=640,
        title='Verandering in Democratie Rang (2019-2023)',
        geo={'showocean': True, 'oceancolor': '#a8d5f2', 'landcolor': '#ffffff'},
        margin={'t': 80, 'r': 20, 'b': 80, 'l': 20},
    )
)
 
fig.update_geos(showcountries=True, showcoastlines=False)
 
fig.show()

In [ ]:
# Creëer een wereldkaart om de verschillen tussen de veranderingen van persvrijheid en democratie te laten zien

# Definieer kleuren op basis van categorieën
def get_color(verandering_pers, verandering_dem):
    if verandering_pers == 0 and verandering_dem == 0:
        return 0  # Blauw: beide veranderingen zijn 0
    elif (verandering_pers >= 0 and verandering_dem > 0) or (verandering_pers > 0 and verandering_dem >= 0):
        return 1  # Groen: beide veranderingen zijn positief
    elif verandering_pers < 0 and verandering_dem < 0:
        return 2  # Rood: beide veranderingen zijn negatief
    elif verandering_pers < 0 and verandering_dem >= 0:
        return 3  # Geel: persvrijheid is negatief, democratie is positief
    elif verandering_pers >= 0 and verandering_dem < 0:
        return 4  # Oranje: persvrijheid is positief, democratie is negatief

# Voeg kleur toe aan DataFrame
eind_df['color'] = eind_df.apply(lambda row: get_color(row['persvrijheid_verandering'], row['democratie_verandering']), axis=1)
 
# Maak een lijst met kleuren op basis van categorie
colorscale = [
    'blue',    # Blauw: beide veranderingen zijn 0
    '#90ee90', # Groen: beide veranderingen zijn positief
    'red',     # Rood: beide veranderingen zijn negatief
    'yellow',  # Geel: persvrijheid is negatief, democratie is positief
    'orange',  # Oranje: persvrijheid is positief, democratie is negatief
]
 
# Creëer de choropleth trace
data = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['color'],  # Gebruik de 'color' kolom als numerieke waarde voor de kleuren
    hoverinfo='location+text',  # Toon locatie, kleurwaarde en tekst bij hover    
    text=eind_df.apply(lambda row: f'Persvrijheid: {row["persvrijheid_verandering"]}<br>Democratie: {row["democratie_verandering"]}', axis=1),
    marker_line_color='gray',  # Voeg een rand toe aan de markeringen
    colorscale=colorscale,
    showscale=False,  # Zet showscale uit om geen kleurenschaal te tonen
)
 
# Creëer de layout
layout = go.Layout(
    width=900,
    height=600,
    title='Verandering in Persvrijheid en Democratische Rang (2019-2023)',
    geo=dict(
        showocean=True,
        oceancolor='#a8d5f2',
        showcountries=True,
        countrycolor='white',
        landcolor='white',
        showland=True,
        projection_type='mercator'
    ),
    margin={'t': 50, 'r': 90, 'b': 50, 'l': 40},
)
 
# Voeg de legenda handmatig toe als annotaties
legend_items = {
    'Pers en Demo neutraal': 0,
    'Pers en Demo positief': 1,
    'Pers en Demo negatief': 2,
    'Pers negatief Demo positief': 3,
    'Pers positief Demo negatief': 4,
}
 
annotations = []
for label, color_index in legend_items.items():
    annotations.append(dict(
        x=0.93,
        y=0.95 - (len(colorscale) - color_index - 1) * 0.05,
        xref='paper',
        yref='paper',
        text=label,
        showarrow=False,
        font=dict(size=10),
        align='left',
        xanchor='left',
    ))

# Voeg de legenda bolletjes toe
for color_index, color_value in enumerate(colorscale):
    annotations.append(dict(
        x=0.92,
        y=0.95 - (len(colorscale) - color_index - 1) * 0.05,
        xref='paper',
        yref='paper',
        text = '',
        showarrow=False,
        bgcolor=colorscale[color_index],
        width=13,
        height=13,
        borderpad=0,
    ))

layout['annotations'] = annotations
 
# Maak de figuur
fig = go.Figure(data=[data], layout=layout)
 
# Toon de figuur
fig.show()

In [ ]:
# Creëer een wereldkaart om de verschillen tussen de veranderingen van persvrijheid en conflicten te laten zien

# Definieer kleuren op basis van categorieën
def get_color(verandering_pers, verandering_conf):
    if verandering_pers == 0 and verandering_conf == 0:
        return 'Pers en Demo neutraal'  # Blauw: beide veranderingen zijn 0
    elif (verandering_pers >= 0 and verandering_conf > 0) or (verandering_pers > 0 and verandering_conf >= 0):
        return 'Pers en Demo positief'  # Groen: beide veranderingen zijn positief
    elif verandering_pers < 0 and verandering_conf < 0:
        return 'Pers en Demo negatief'  # Rood: beide veranderingen zijn negatief
    elif verandering_pers < 0 and verandering_conf >= 0:
        return 'Pers negatief Demo positief'  # Geel: persvrijheid is negatief, conflict is positief
    elif verandering_pers >= 0 and verandering_conf < 0:
        return 'Pers positief Demo negatief'  # Oranje: persvrijheid is positief, conflict is negatief

# Voeg kleur toe aan DataFrame
eind_df['color'] = eind_df.apply(lambda row: get_color(row['persvrijheid_verandering'], row['conflict_verandering']), axis=1)

# Maak een lijst met kleuren op basis van categorie
color_discrete_map = {
    'Pers en Demo neutraal': 'blue',    
    'Pers en Demo positief': '#90ee90',
    'Pers en Demo negatief': 'red',     
    'Pers negatief Demo positief': 'yellow',
    'Pers positief Demo negatief': 'orange'
}

# Maak de choropleth map met Plotly Express
fig = px.choropleth(
    eind_df,
    locations="land",
    locationmode="country names",
    color="color",
    hover_name="land",
    hover_data={
        "color": False,
        "persvrijheid_verandering": True,
        "conflict_verandering": True
    },
    color_discrete_map=color_discrete_map,
    title="Verandering in Persvrijheid en Conflict Rang (2019-2023)"
)

# Aanpassen van de layout van de figuur
fig.update_geos(
    showocean=True,
    oceancolor='#a8d5f2',
    showcountries=True,
    countrycolor='white',
    landcolor='white',
    showland=True,
    projection_type='mercator'
)

fig.update_layout(
    width=900,
    height=600,
    margin={'t': 80, 'r': 80, 'b': 50, 'l': 80},
    title={
        'text': "Verandering in Persvrijheid en Conflict Rang (2019-2023)",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)

# Toon de figuur
fig.show()
